# Workbook 3 Lasso
After selecting my own feature, I then used Lasso to see what features it selected and compared the scores and the coefficients

In [129]:
import numpy as np
import pandas as pd
import patsy
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.model_selection import cross_val_score
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.feature_selection import SelectKBest

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [130]:
train = pd.read_csv('./train.csv')

In [131]:
test = pd.read_csv('./test.csv')

In [132]:
y = train['SalePrice']
X = train.drop(['SalePrice'], axis = "columns")

In [133]:
test_id = test['Id']

In [134]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)

In [135]:
def clean_me(df):
    #Replaces Null values outside of object columns 
    for i in range(len(df.columns)): 
        if [df.iloc[: ,i].dtypes!=object] == True:
            mean = df.iloc[: ,i].mean()
            df.iloc[: ,i].fillna(mean, inplace=True)
        else:
            pass
        #Replaces other null values with AANone
    str_cols = df.columns[df.dtypes==object]
    df[str_cols] = df[str_cols].fillna('AANone')
    df.fillna(0,inplace=True)
        #Change selected columns from Int to float
    for col in ['Low Qual Fin SF', 'Gr Liv Area', 'Lot Area', '1st Flr SF', '2nd Flr SF',
                'Wood Deck SF', 'Open Porch SF', 'Enclosed Porch', '3Ssn Porch', 'Screen Porch', 'Pool Area', 'Misc Val']:
                df[col] = df[col].astype('float64')
        #Makes remaining int colomns objects 
    int_cols = df.columns[df.dtypes=='int64']
    df[int_cols] = df[int_cols].astype('object')
    df = df.drop(['Id', 'PID'], axis=1)
    return df

In [136]:
X_train = clean_me(X_train)

C:\Users\jmvail\Anaconda3\envs\dsi\lib\site-packages\pandas\core\frame.py:2540: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]
C:\Users\jmvail\Anaconda3\envs\dsi\lib\site-packages\pandas\core\frame.py:3035: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)
C:\Users\jmvail\Anaconda3\envs\dsi\lib\site-packages\ipykernel\__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-doc

In [137]:
X_test = clean_me(X_test)

C:\Users\jmvail\Anaconda3\envs\dsi\lib\site-packages\pandas\core\frame.py:2540: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]
C:\Users\jmvail\Anaconda3\envs\dsi\lib\site-packages\pandas\core\frame.py:3035: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)
C:\Users\jmvail\Anaconda3\envs\dsi\lib\site-packages\ipykernel\__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-doc

In [138]:
test = clean_me(test)

**Cleaning Done**

Used lasso with all the variables to pair down the coefficience

In [139]:
def get_dummied(train, test):
    list = []
    
    for i in train.columns:
        if train[i].dtype == object:
            list.append(i)
            #puts all object names in list
    full_data = pd.concat([train, test], axis=0)
    #puts training data and test data in one dataframe
    full_data = pd.get_dummies(full_data, columns=list, drop_first=True)
    #applies get dummy to whole list
    X_dummied = full_data[:len(train)]
    test_dummied = full_data.iloc[-len(test):]
    #splits full data back into 2 data frames
    
    return X_dummied, test_dummied

In [140]:
train_dum, test_dum = get_dummied(X_train, X_test)

In [141]:
 ss = StandardScaler()

In [142]:
train_scaled = ss.fit_transform(train_dum)

**NOTE**

should be transforming on fit done on train. Will be fixed in future updates

In [143]:
test_scaled = ss.fit_transform(test_dum)

In [144]:
#l_alphas = np.arange(0.001, 0.2, 0.0001) 
lasso_m = LassoCV( cv=3)
lasso_m =lasso_m.fit(train_scaled, y_train)
lasso_optimal_alpha = lasso_m.alpha_
print(lasso_optimal_alpha,':is the optimal alpha')
print(lasso_m.score(train_scaled, y_train))
print(lasso_m.score(test_scaled, y_test))

785.3199159003299 :is the optimal alpha
0.9493925371026144
0.8487029208762521


In [145]:
coef = pd.DataFrame(train_dum.columns, lasso_m.coef_, columns=['Feature'])
coef['abs'] = abs(coef.index)
coef['coef']= coef.index
coef = coef.sort_values(['abs'], ascending=False).head(25)
coef

,Feature,abs,coef
22255.842675,Gr Liv Area,22255.842675,22255.842675
-16336.843676,TotRms AbvGrd_15,16336.843676,-16336.843676
10887.604652,Overall Qual_9,10887.604652,10887.604652
10144.379525,Overall Qual_10,10144.379525,10144.379525
8853.051633,BsmtFin SF 1,8853.051633,8853.051633
8273.034058,Overall Qual_8,8273.034058,8273.034058
6322.727955,Total Bsmt SF,6322.727955,6322.727955
5870.987630,Lot Area,5870.987630,5870.987630
5216.813105,Neighborhood_NridgHt,5216.813105,5216.813105
5183.351953,Garage Area,5183.351953,5183.351953


In [146]:
lasso_m.coef_

array([ 1.75572444e+02,  5.87098763e+03,  3.82247281e+03,  8.85305163e+03,
        0.00000000e+00, -0.00000000e+00,  6.32272795e+03,  2.96153112e+03,
        0.00000000e+00, -7.12804959e+02,  2.22558427e+04,  3.52424450e+02,
       -0.00000000e+00, -0.00000000e+00,  7.98400099e+02,  5.18335195e+03,
        6.21256981e+02,  1.47381166e+03, -0.00000000e+00,  0.00000000e+00,
        1.81447655e+03, -0.00000000e+00, -0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00, -0.00000000e+00,  4.48038195e+02,
       -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,  0.00000000e+00,
       -0.00000000e+00, -2.59347867e+00,  0.00000000e+00, -0.00000000e+00,
       -0.00000000e+00, -0.00000000e+00, -9.44195781e+02,  9.91967709e+02,
        0.00000000e+00, -0.00000000e+00,  0.00000000e+00, -1.52566471e+03,
        0.00000000e+00, -0.00000000e+00, -2.79517545e+02,  0.00000000e+00,
       -0.00000000e+00, -0.00000000e+00,  2.11923289e+03, -0.00000000e+00,
        0.00000000e+00, -

In [147]:
train_dum, pred_dum = get_dummied(X_train, test)

In [148]:
train_scaled = ss.fit_transform(train_dum)

In [149]:
pred_scaled = ss.fit_transform(pred_dum)

In [150]:
lasso_m = LassoCV( cv=3)
lasso_m =lasso_m.fit(train_scaled, y_train)
lasso_optimal_alpha = lasso_m.alpha_

In [151]:
y_hat = lasso_m.predict(pred_scaled)

In [152]:
#scale_trial_2 = pd.DataFrame(y_hat, index = test_id)
#scale_trial_2.columns = ['SalePrice']
#scale_trial_2.head()

In [153]:
#scale_trial_2.to_csv("./scale_trial_2.csv")

Used good coefficients and used poly features and lassoed again. Did not work

In [154]:
coefs = pd.DataFrame([lasso_m.coef_], columns= train_dum.columns)
coefs = coefs.T

In [155]:
mask = coefs[0] > 0
good = coefs[mask]
good = good.T

In [156]:
pd.options.display.max_rows = 999

In [157]:
pd.options.display.max_rows

999

Coefficient List (most important features)

In [158]:
good.T

,0
Lot Frontage,165.734609
Lot Area,5892.827730
Mas Vnr Area,3834.831680
BsmtFin SF 1,8887.521499
Total Bsmt SF,6290.306423
1st Flr SF,2888.370016
Gr Liv Area,22394.665500
Bsmt Full Bath,364.094393
Garage Cars,983.089643
Garage Area,5088.840796


In [159]:
good.columns

Index(['Lot Frontage', 'Lot Area', 'Mas Vnr Area', 'BsmtFin SF 1',
       'Total Bsmt SF', '1st Flr SF', 'Gr Liv Area', 'Bsmt Full Bath',
       'Garage Cars', 'Garage Area', 'Wood Deck SF', 'Open Porch SF',
       'Screen Porch', 'MS SubClass_60', 'MS Zoning_FV', 'Land Contour_HLS',
       'Lot Config_CulDSac', 'Land Slope_Sev', 'Neighborhood_Crawfor',
       'Neighborhood_GrnHill', 'Neighborhood_NoRidge', 'Neighborhood_NridgHt',
       'Neighborhood_StoneBr', 'Condition 1_Norm', 'Condition 1_PosA',
       'Condition 1_PosN', 'Condition 2_PosN', 'Overall Qual_7',
       'Overall Qual_8', 'Overall Qual_9', 'Overall Qual_10', 'Overall Cond_7',
       'Overall Cond_8', 'Year Built_1892', 'Year Built_1901',
       'Year Built_1935', 'Year Built_1945', 'Year Built_1974',
       'Year Built_1979', 'Year Built_2000', 'Year Built_2005',
       'Year Built_2006', 'Year Built_2008', 'Year Built_2009',
       'Year Remod/Add_1986', 'Year Remod/Add_1990', 'Year Remod/Add_2001',
       'Year Remod

In [160]:
lasso_col = ['Lot Area', 'Mas Vnr Area', 'BsmtFin SF 1', 'BsmtFin SF 2',
       'Total Bsmt SF', '1st Flr SF', 'Gr Liv Area', 'Bsmt Full Bath',
       'Garage Cars', 'Wood Deck SF', 'Open Porch SF', '3Ssn Porch',
       'Screen Porch', 'MS Zoning_FV', 'Lot Shape_IR2', 'Land Contour_HLS',
       'Lot Config_CulDSac', 'Neighborhood_ClearCr', 'Neighborhood_Crawfor',
       'Neighborhood_NoRidge', 'Neighborhood_NridgHt', 'Neighborhood_Somerst',
       'Neighborhood_StoneBr', 'Neighborhood_Veenker', 'Condition 1_Norm',
       'Condition 1_PosA', 'Condition 1_PosN', 'Overall Qual_7',
       'Overall Qual_8', 'Overall Qual_9', 'Overall Qual_10', 'Overall Cond_7',
       'Overall Cond_8', 'Overall Cond_9', 'Year Built_1892',
       'Year Built_1932', 'Year Built_1991', 'Year Built_1998',
       'Year Built_2006', 'Year Built_2008', 'Year Built_2009',
       'Year Remod/Add_1996', 'Year Remod/Add_2001', 'Year Remod/Add_2006',
       'Year Remod/Add_2007', 'Year Remod/Add_2008', 'Year Remod/Add_2010',
       'Roof Matl_Membran', 'Exterior 1st_BrkFace', 'Exterior 1st_CemntBd',
       'Exterior 1st_VinylSd', 'Exterior 2nd_VinylSd', 'Mas Vnr Type_Stone',
       'Foundation_PConc', 'Bsmt Qual_Ex', 'Bsmt Qual_Gd', 'Bsmt Exposure_Gd',
       'BsmtFin Type 1_ALQ', 'BsmtFin Type 1_GLQ', 'Full Bath_3',
       'Half Bath_1', 'Bedroom AbvGr_4', 'Kitchen AbvGr_1', 'TotRms AbvGrd_8',
       'TotRms AbvGrd_9', 'TotRms AbvGrd_10', 'TotRms AbvGrd_11',
       'Functional_Typ', 'Fireplaces_1', 'Fireplaces_2', 'Fireplaces_4',
       'Fireplace Qu_Gd', 'Fireplace Qu_TA', 'Garage Type_Attchd',
       'Garage Finish_Fin', 'Garage Qual_Ex', 'Garage Qual_Gd',
       'Paved Drive_Y', 'Sale Type_Con', 'Sale Type_New', 'Sale Type_Oth']

# Attempt made on second Lasso,
Tried to use polyfeatures on top 50 and lasso 

In [161]:
train_dum, test_dum = get_dummied(X_train, X_test)

In [162]:
train_good = train_dum[lasso_col]

In [163]:
test_good = test_dum[lasso_col]

In [164]:
train_scaled_good = ss.fit_transform(train_good)

In [165]:
test_scaled_good = ss.fit_transform(test_good)

In [166]:
poly = PolynomialFeatures(include_bias = True)

In [167]:
train_scaled_poly_good = poly.fit_transform(train_scaled_good)

In [168]:
test_scaled_poly_good = poly.fit_transform(test_scaled_good)

In [169]:
lasso_m = LassoCV( cv=3)
lasso_m =lasso_m.fit(train_scaled_poly_good, y_train)
lasso_optimal_alpha = lasso_m.alpha_
print(lasso_optimal_alpha,':is the optimal alpha')

print(lasso_m.score(test_scaled_poly_good, y_test))

836.4032511144579 :is the optimal alpha
0.889727380428517


In [170]:
lasso_m.coef_

array([   0.        , 5653.804842  ,    0.        , ...,  216.38333072,
         -0.        ,   83.02392746])

In [171]:
good

,Lot Frontage,Lot Area,Mas Vnr Area,BsmtFin SF 1,Total Bsmt SF,1st Flr SF,Gr Liv Area,Bsmt Full Bath,Garage Cars,Garage Area,...,Fireplace Qu_Gd,Garage Type_BuiltIn,Garage Finish_Fin,Garage Qual_Gd,Paved Drive_Y,Misc Feature_Othr,Mo Sold_6,Sale Type_Con,Sale Type_New,Sale Type_Oth
0,165.734609,5892.82773,3834.83168,8887.521499,6290.306423,2888.370016,22394.6655,364.094393,983.089643,5088.840796,...,1964.932075,36.739786,2722.472209,1752.770591,1252.209845,766.867145,53.164161,503.60893,2096.425444,544.419405
